# Twitter Sentiment Analysis.

## Business Problem.

We want to analyze a COVID-19 Twitter dataset to understand how positive and negative trends spread after news announcements. Additionally, we want to use a bot detection algorithm to determine what percentage of each sentiment is made up of bots and how this affects the general sentiment of the public (non-bots).

Key Questions:

How do positive and negative sentiments spread among users after a news announcement related to COVID-19?
What proportion of tweets in each sentiment category (positive/negative/neutral) come from bots?
How do bots influence the general sentiment of non-bot users?
Purpose:

Help media outlets measure the impact of their announcements on public sentiment.
Assist public health agencies in identifying misinformation or bot-driven content to improve communication strategies.
Support social media platforms in detecting and limiting bot activity that could distort public opinion.
Goals:

Track sentiment trends over time.
Quantify bot participation in each sentiment category.
Measure the influence of bots on genuine public sentiment.

## Datasets.

1. [A Twitter Dataset of 40+ million tweets related to COVID-19.](https://zenodo.org/records/3723940)

2. [Twitter Bots Accounts.](https://www.kaggle.com/datasets/davidmartngutirrez/twitter-bots-accounts)


## Installing the required modules.

We'll start with installing the requirements [available here](https://github.com/leksea/capstone-twitter-sentiment-analysis/blob/main/requirements.txt).

In [12]:
!wget https://raw.githubusercontent.com/leksea/capstone-twitter-sentiment-analysis/main/requirements.txt
!pip install -r 'requirements.txt'

--2024-12-24 21:07:01--  https://raw.githubusercontent.com/leksea/capstone-twitter-sentiment-analysis/main/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 98 [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[===================>]      98  --.-KB/s    in 0s      

2024-12-24 21:07:01 (3.30 MB/s) - ‘requirements.txt’ saved [98/98]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 61.8 MB/s eta 0:00:00


### Importing modules.

In [16]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
import string
import re
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import branca.colormap as cm
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import requests
import folium
from folium import plugins
from folium.plugins import HeatMap
import branca.colormap
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk import pos_tag, ne_chunk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from wordcloud import WordCloud
from tqdm import tqdm, notebook
%matplotlib inline
# stop words for tokenizer
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Loading The Dataset.